# $U(1)$ Gauge Model using L2HMC in graph mode

--------------------------------------------------------------------
### TODO:
* [ ] Look at performance on Cooley (longer training runs).
* [ ] Fit observables to Eq. \ref{eq:therm_time} to determine the thermalization time $\tau$.

\begin{equation} 
f(t) \equiv A \exp^{-t / \tau}+ \,\, B
\label{eq:therm_time}
\end{equation}
* [ ] Look at defining a distance metric as the difference in topological charge between two samples and see what effect adding this as an additional term to the loss function has on the models' ability to tunnel between topological sectors.

--------------------------------------------------------------------

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2
%matplotlib notebook

#from l2hmc_eager import gauge_dynamics_eager as gde

In [2]:
#tf.enable_eager_execution()
tfe = tf.contrib.eager

#from u1_model_eager import GaugeModelEager, train_one_iter
from gauge_model import GaugeModel

import utils.gauge_model_helpers as helpers

Using TensorFlow backend.


In [3]:
def autocorr(x):
    result = np.correlate(x, x, mode='full')
    result /= result[result.argmax()]
    return result[result.size//2:]

## Define parameters

In [13]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 1.,
    'beta_init': 1.,
    'beta_final': 8.,
    'num_samples': 5,
    'num_steps': 5,
    'eps': 0.2,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 5e-4,
    'learning_rate_decay_steps': 250,
    'learning_rate_decay_rate': 0.98,
    'train_steps': 6000,
    'save_steps': 500,
    'logging_steps': 50,
    'annealing_steps': 100,
    'annealing_factor': 0.96,
    'clip_value': 1e6,
    'rand': False,
    'metric': 'l2',
}
#tf.reset_default_graph()

## Create model

In [14]:
#del model

In [17]:
config = tf.ConfigProto()
tf.reset_default_graph()

In [18]:
model = GaugeModel(params=params,
                   config=config,
                   sess=None,
                   conv_net=True,
                   hmc=False,
                   log_dir=None,
                   restore=False,
                   eps_trainable=True,
                   aux=False,
                   annealing=True,
                   clip_grads=False)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_100/
log_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_100/
info_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_100/run_info/
figs_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_100/figures/
conv_net: True
hmc: False
aux: False
eps_trainable: True
clip_grads: False
annealing: True
beta: 1.0
time_size: 8
space_size: 8
link_type: U1
dim: 2
beta_init: 1.0
beta_final: 8.0
num_samples: 5
num_steps: 5
eps: 0.2
loss_scale: 0.1
loss_eps: 0.0001
learning_rate_init: 0.0005
learning_rate_decay_steps: 250
learning_rate_decay_rate: 0.98
train_steps: 6000
save_steps: 500
logging_steps: 50
annealing_steps: 100
annealing_factor: 0.96
clip_value: 1000000.0
rand: False
metric: l2
samples_history_dir: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_100/samples_history
--------------------------------------------------------------------------------

Building graph... (started at: Tue Jan 15 12:56:55 2019)
  Creat

In [19]:
model.train(model.train_steps, kill_sess=False)

----------------------------------------------------------------------------------------------------
     STEP           LOSS       NORM. TIME     ACCEPT %        EPS           BETA           LR      
----------------------------------------------------------------------------------------------------
     0/6000     -1.023e+04      0.1176        0.4989          0.2            1          0.0005    
Time to complete logging: 120.9 

     1/6000     -1.434e+04     0.0001431         1          0.1995           1          0.0005    
     2/6000     -1.069e+04     0.0001351         1           0.199           1          0.0005    
     3/6000        -8558       0.0001421         1          0.1986           1          0.0005    
     4/6000        -7199       0.0001404         1          0.1981           1          0.0005    
     5/6000        -7394       0.0001298         1          0.1977           1          0.0005    
     6/6000        -6269       0.0001411      0.8861        0.1974    

    80/6000        -5281       0.0001252         1          0.1801           1          0.0005    
    81/6000        -5372       0.0001097         1           0.18            1          0.0005    
    82/6000        -5161       0.0001573         1           0.18            1          0.0005    
    83/6000        -4796       0.0001323         1          0.1799           1          0.0005    
    84/6000        -4922       0.0001563         1          0.1799           1          0.0005    
    85/6000        -5564       0.0001367         1          0.1799           1          0.0005    
    86/6000        -5534       0.0001859         1          0.1798           1          0.0005    
    87/6000        -4962       0.0001837         1          0.1798           1          0.0005    
    88/6000        -5184       0.0001602      0.9199        0.1798           1          0.0005    
    89/6000        -5387       0.0001264         1          0.1797           1          0.0005    
    90/600

   163/6000        -5028       0.0001433         1          0.1751         1.042        0.0005    
   164/6000        -4493       0.000134          1           0.175         1.042        0.0005    
   165/6000        -4403       0.0001232         1           0.175         1.042        0.0005    
   166/6000        -4111       0.0001358         1          0.1749         1.042        0.0005    
   167/6000        -4691       0.0001293         1          0.1749         1.042        0.0005    
   168/6000        -5309       0.0001363         1          0.1749         1.042        0.0005    
   169/6000        -4908       0.0001213         1          0.1749         1.042        0.0005    
   170/6000        -4117       0.0001444         1          0.1748         1.042        0.0005    
   171/6000        -4487       0.0001389         1          0.1748         1.042        0.0005    
   172/6000        -4618       0.0001501         1          0.1748         1.042        0.0005    
   173/600

   246/6000        -4686       0.0001194      0.9467        0.1715         1.085        0.0005    
   247/6000        -4445       0.0001101         1          0.1715         1.085        0.0005    
   248/6000        -4536       0.0001319         1          0.1714         1.085        0.0005    
   249/6000        -5018       0.0001355         1          0.1714         1.085        0.0005    
Time to complete logging: 0.161 

   250/6000        -4407       0.0001353         1          0.1714         1.085        0.0005    
   251/6000        -4093       0.0001156         1          0.1714         1.085        0.0005    
   252/6000        -4706       0.0001371         1          0.1714         1.085        0.0005    
   253/6000        -5378       0.0001271         1          0.1714         1.085        0.0005    
   254/6000        -4406       0.0001563         1          0.1714         1.085        0.0005    
   255/6000        -4265       0.0001364         1          0.1714         

   329/6000        -4912       0.0001318         1          0.1684         1.13         0.0005    
   330/6000        -4941       0.0001293         1          0.1683         1.13         0.0005    
   331/6000        -4905       0.0001441         1          0.1683         1.13         0.0005    
   332/6000        -5364       0.0001655         1          0.1682         1.13         0.0005    
   333/6000        -4276       0.0001217         1          0.1681         1.13         0.0005    
   334/6000        -4633       0.0001271         1          0.1681         1.13         0.0005    
   335/6000        -4451       0.0001228      0.9938         0.168         1.13         0.0005    
   336/6000        -4743       0.0001247         1          0.1679         1.13         0.0005    
   337/6000        -4548       0.0001296         1          0.1679         1.13         0.0005    
   338/6000        -4874       0.0001288         1          0.1678         1.13         0.0005    
   339/600

...

  5672/6000       -313.3       0.0001361         1         0.004052          8          0.0005    
  5673/6000       -235.1       0.0001248         1         0.004102          8          0.0005    
  5674/6000       -313.1       0.0001358         1         0.004166          8          0.0005    
  5675/6000       -391.5       0.0001307         1         0.004218          8          0.0005    
  5676/6000       -471.7       0.0001412         1         0.004261          8          0.0005    
  5677/6000       -313.2       0.0001307         1         0.004293          8          0.0005    
  5678/6000       -469.3       0.0001425         1         0.004324          8          0.0005    
  5679/6000        1.089       0.0001417      0.9976       0.004304          8          0.0005    
  5680/6000       -312.5       0.0001362      0.9991       0.004361          8          0.0005    
  5681/6000       -154.8       0.0001362      0.9974        0.00439          8          0.0005    
  5682/600

  5755/6000       -76.79       0.0002866         1          0.00351          8          0.0005    
  5756/6000       -234.8       0.0001602         1         0.003542          8          0.0005    
  5757/6000       -313.7       0.000203          1         0.003627          8          0.0005    
  5758/6000       -233.7       0.0001843         1         0.003725          8          0.0005    
  5759/6000       -76.77       0.0001533       0.997       0.003814          8          0.0005    
  5760/6000       -313.1       0.0001642         1          0.00396          8          0.0005    
  5761/6000        1.488       0.0001602         1         0.004092          8          0.0005    
  5762/6000       -546.4       0.0001478      0.9972       0.004299          8          0.0005    
  5763/6000        -469        0.0001439      0.9945       0.004405          8          0.0005    
  5764/6000        -469        0.0001583         1         0.004441          8          0.0005    
  5765/600

  5838/6000       -469.7       0.0001686         1         0.003852          8          0.0005    
  5839/6000       -470.2       0.0001442         1         0.003728          8          0.0005    
  5840/6000        1.985       0.0001386         1         0.003581          8          0.0005    
  5841/6000       -233.7       0.0001543         1         0.003565          8          0.0005    
  5842/6000       -233.8       0.0001453         1         0.003582          8          0.0005    
  5843/6000       -76.94       0.0001431         1         0.003641          8          0.0005    
  5844/6000        -392        0.0001486         1         0.003776          8          0.0005    
  5845/6000       -543.4       0.0001374      0.9972       0.003905          8          0.0005    
  5846/6000       -312.4       0.0001392      0.9998        0.00387          8          0.0005    
  5847/6000       -628.4       0.0001557         1         0.003875          8          0.0005    
  5848/600

  5921/6000       -704.7       0.0001311         1         0.004208          8          0.0005    
  5922/6000        -392        0.0001595      0.9999       0.004182          8          0.0005    
  5923/6000       -390.9       0.0001401         1         0.004138          8          0.0005    
  5924/6000       -469.3       0.0001496         1          0.00408          8          0.0005    
  5925/6000       -313.2       0.0001308         1         0.003983          8          0.0005    
  5926/6000       -155.8       0.0001406      0.9958       0.003905          8          0.0005    
  5927/6000       -152.2       0.0001466      0.9958       0.003872          8          0.0005    
  5928/6000        -313        0.0001501      0.9993        0.00382          8          0.0005    
  5929/6000       -155.7       0.000139          1         0.003789          8          0.0005    
  5930/6000       -313.2       0.0001414         1         0.003809          8          0.0005    
  5931/600

Time to complete logging: 0.3686

Training complete!
Saving checkpoint to: /Users/saforem2/ANL/l2hmc/gauge_logs_graph/run_100/model.ckpt



In [52]:
run_steps = [100, 200, 500, 1000, 5000, 10000]

In [53]:
model.beta_final

8.0

In [54]:
for steps in run_steps:
    _ = model.run(steps)

Running (trained) L2HMC sampler for 100 steps...
step:   0   /100 time/step/sample/link: 0.04557
step:   1   /100 time/step/sample/link: 3.695e-05
step:   2   /100 time/step/sample/link: 4.872e-05
step:   3   /100 time/step/sample/link: 3.788e-05
step:   4   /100 time/step/sample/link: 5.051e-05
step:   5   /100 time/step/sample/link: 3.762e-05
step:   6   /100 time/step/sample/link: 3.635e-05
step:   7   /100 time/step/sample/link: 3.462e-05
step:   8   /100 time/step/sample/link: 3.862e-05
step:   9   /100 time/step/sample/link: 4.165e-05
step:   10  /100 time/step/sample/link: 4.956e-05
step:   11  /100 time/step/sample/link: 4.79e-05
step:   12  /100 time/step/sample/link: 4.412e-05
step:   13  /100 time/step/sample/link: 3.72e-05
step:   14  /100 time/step/sample/link: 3.82e-05
step:   15  /100 time/step/sample/link: 3.77e-05
step:   16  /100 time/step/sample/link: 4.309e-05
step:   17  /100 time/step/sample/link: 4.092e-05
step:   18  /100 time/step/sample/link: 4.171e-05
step:  

step:   60  /200 time/step/sample/link: 4.479e-05
step:   61  /200 time/step/sample/link: 3.688e-05
step:   62  /200 time/step/sample/link: 4.009e-05
step:   63  /200 time/step/sample/link: 3.595e-05
step:   64  /200 time/step/sample/link: 4.524e-05
step:   65  /200 time/step/sample/link: 4.842e-05
step:   66  /200 time/step/sample/link: 4.408e-05
step:   67  /200 time/step/sample/link: 4.8e-05
step:   68  /200 time/step/sample/link: 3.873e-05
step:   69  /200 time/step/sample/link: 3.787e-05
step:   70  /200 time/step/sample/link: 4.194e-05
step:   71  /200 time/step/sample/link: 3.943e-05
step:   72  /200 time/step/sample/link: 4.137e-05
step:   73  /200 time/step/sample/link: 3.173e-05
step:   74  /200 time/step/sample/link: 3.728e-05
step:   75  /200 time/step/sample/link: 5.278e-05
step:   76  /200 time/step/sample/link: 4.652e-05
step:   77  /200 time/step/sample/link: 3.609e-05
step:   78  /200 time/step/sample/link: 3.857e-05
step:   79  /200 time/step/sample/link: 3.935e-05
st

step:   20  /500 time/step/sample/link: 4.313e-05
step:   21  /500 time/step/sample/link: 4.793e-05
step:   22  /500 time/step/sample/link: 4.605e-05
step:   23  /500 time/step/sample/link: 3.907e-05
step:   24  /500 time/step/sample/link: 4.244e-05
step:   25  /500 time/step/sample/link: 4.54e-05
step:   26  /500 time/step/sample/link: 4.12e-05
step:   27  /500 time/step/sample/link: 4.191e-05
step:   28  /500 time/step/sample/link: 4.353e-05
step:   29  /500 time/step/sample/link: 4.288e-05
step:   30  /500 time/step/sample/link: 4.515e-05
step:   31  /500 time/step/sample/link: 4.304e-05
step:   32  /500 time/step/sample/link: 4.571e-05
step:   33  /500 time/step/sample/link: 3.944e-05
step:   34  /500 time/step/sample/link: 4.152e-05
step:   35  /500 time/step/sample/link: 4.291e-05
step:   36  /500 time/step/sample/link: 4.092e-05
step:   37  /500 time/step/sample/link: 4.325e-05
step:   38  /500 time/step/sample/link: 5.315e-05
step:   39  /500 time/step/sample/link: 5.055e-05
st

step:  186  /500 time/step/sample/link: 4.215e-05
step:  187  /500 time/step/sample/link: 4.437e-05
step:  188  /500 time/step/sample/link: 4.792e-05
step:  189  /500 time/step/sample/link: 4.14e-05
step:  190  /500 time/step/sample/link: 6.799e-05
step:  191  /500 time/step/sample/link: 4.552e-05
step:  192  /500 time/step/sample/link: 4.459e-05
step:  193  /500 time/step/sample/link: 3.806e-05
step:  194  /500 time/step/sample/link: 3.553e-05
step:  195  /500 time/step/sample/link: 3.71e-05
step:  196  /500 time/step/sample/link: 7.158e-05
step:  197  /500 time/step/sample/link: 7.398e-05
step:  198  /500 time/step/sample/link: 5.56e-05
step:  199  /500 time/step/sample/link: 5.043e-05
step:  200  /500 time/step/sample/link: 4.037e-05
step:  201  /500 time/step/sample/link: 6.982e-05
step:  202  /500 time/step/sample/link: 8.551e-05
step:  203  /500 time/step/sample/link: 6.383e-05
step:  204  /500 time/step/sample/link: 7.644e-05
step:  205  /500 time/step/sample/link: 6.726e-05
ste

step:  351  /500 time/step/sample/link: 9.9e-05
step:  352  /500 time/step/sample/link: 9.172e-05
step:  353  /500 time/step/sample/link: 9.876e-05
step:  354  /500 time/step/sample/link: 6.824e-05
step:  355  /500 time/step/sample/link: 6.243e-05
step:  356  /500 time/step/sample/link: 7.108e-05
step:  357  /500 time/step/sample/link: 6.804e-05
step:  358  /500 time/step/sample/link: 8.664e-05
step:  359  /500 time/step/sample/link: 5.476e-05
step:  360  /500 time/step/sample/link: 7.243e-05
step:  361  /500 time/step/sample/link: 4.448e-05
step:  362  /500 time/step/sample/link: 4.452e-05
step:  363  /500 time/step/sample/link: 6.807e-05
step:  364  /500 time/step/sample/link: 9.004e-05
step:  365  /500 time/step/sample/link: 5.476e-05
step:  366  /500 time/step/sample/link: 7.999e-05
step:  367  /500 time/step/sample/link: 7.846e-05
step:  368  /500 time/step/sample/link: 6.267e-05
step:  369  /500 time/step/sample/link: 7.85e-05
step:  370  /500 time/step/sample/link: 8.345e-05
ste

...

step:  9338 /10000 time/step/sample/link: 4.389e-05
step:  9339 /10000 time/step/sample/link: 3.855e-05
step:  9340 /10000 time/step/sample/link: 4.177e-05
step:  9341 /10000 time/step/sample/link: 4.012e-05
step:  9342 /10000 time/step/sample/link: 5.713e-05
step:  9343 /10000 time/step/sample/link: 4.524e-05
step:  9344 /10000 time/step/sample/link: 4.651e-05
step:  9345 /10000 time/step/sample/link: 4.378e-05
step:  9346 /10000 time/step/sample/link: 4.554e-05
step:  9347 /10000 time/step/sample/link: 4.124e-05
step:  9348 /10000 time/step/sample/link: 4.402e-05
step:  9349 /10000 time/step/sample/link: 4.29e-05
step:  9350 /10000 time/step/sample/link: 4.467e-05
step:  9351 /10000 time/step/sample/link: 4.07e-05
step:  9352 /10000 time/step/sample/link: 4.338e-05
step:  9353 /10000 time/step/sample/link: 4.293e-05
step:  9354 /10000 time/step/sample/link: 3.969e-05
step:  9355 /10000 time/step/sample/link: 3.75e-05
step:  9356 /10000 time/step/sample/link: 3.964e-05
step:  9357 /10

step:  9496 /10000 time/step/sample/link: 4.1e-05
step:  9497 /10000 time/step/sample/link: 4.312e-05
step:  9498 /10000 time/step/sample/link: 4.689e-05
step:  9499 /10000 time/step/sample/link: 5.694e-05
step:  9500 /10000 time/step/sample/link: 4.38e-05
step:  9501 /10000 time/step/sample/link: 3.861e-05
step:  9502 /10000 time/step/sample/link: 3.974e-05
step:  9503 /10000 time/step/sample/link: 3.791e-05
step:  9504 /10000 time/step/sample/link: 5.419e-05
step:  9505 /10000 time/step/sample/link: 5.258e-05
step:  9506 /10000 time/step/sample/link: 4.078e-05
step:  9507 /10000 time/step/sample/link: 3.769e-05
step:  9508 /10000 time/step/sample/link: 4.103e-05
step:  9509 /10000 time/step/sample/link: 4.332e-05
step:  9510 /10000 time/step/sample/link: 4.811e-05
step:  9511 /10000 time/step/sample/link: 8.342e-05
step:  9512 /10000 time/step/sample/link: 5.164e-05
step:  9513 /10000 time/step/sample/link: 4.856e-05
step:  9514 /10000 time/step/sample/link: 4.258e-05
step:  9515 /10

step:  9654 /10000 time/step/sample/link: 4.242e-05
step:  9655 /10000 time/step/sample/link: 4.192e-05
step:  9656 /10000 time/step/sample/link: 4.332e-05
step:  9657 /10000 time/step/sample/link: 4.023e-05
step:  9658 /10000 time/step/sample/link: 4.489e-05
step:  9659 /10000 time/step/sample/link: 4.928e-05
step:  9660 /10000 time/step/sample/link: 4.172e-05
step:  9661 /10000 time/step/sample/link: 4.383e-05
step:  9662 /10000 time/step/sample/link: 4.666e-05
step:  9663 /10000 time/step/sample/link: 4.295e-05
step:  9664 /10000 time/step/sample/link: 5.111e-05
step:  9665 /10000 time/step/sample/link: 4.736e-05
step:  9666 /10000 time/step/sample/link: 4.807e-05
step:  9667 /10000 time/step/sample/link: 4.788e-05
step:  9668 /10000 time/step/sample/link: 4.541e-05
step:  9669 /10000 time/step/sample/link: 4.386e-05
step:  9670 /10000 time/step/sample/link: 4.507e-05
step:  9671 /10000 time/step/sample/link: 4.928e-05
step:  9672 /10000 time/step/sample/link: 4.92e-05
step:  9673 /

step:  9813 /10000 time/step/sample/link: 5.077e-05
step:  9814 /10000 time/step/sample/link: 5.563e-05
step:  9815 /10000 time/step/sample/link: 5.178e-05
step:  9816 /10000 time/step/sample/link: 5.08e-05
step:  9817 /10000 time/step/sample/link: 5.504e-05
step:  9818 /10000 time/step/sample/link: 4.414e-05
step:  9819 /10000 time/step/sample/link: 4.805e-05
step:  9820 /10000 time/step/sample/link: 5.036e-05
step:  9821 /10000 time/step/sample/link: 4.904e-05
step:  9822 /10000 time/step/sample/link: 5.237e-05
step:  9823 /10000 time/step/sample/link: 7.376e-05
step:  9824 /10000 time/step/sample/link: 5.286e-05
step:  9825 /10000 time/step/sample/link: 5.464e-05
step:  9826 /10000 time/step/sample/link: 5.207e-05
step:  9827 /10000 time/step/sample/link: 5.717e-05
step:  9828 /10000 time/step/sample/link: 5.688e-05
step:  9829 /10000 time/step/sample/link: 5.683e-05
step:  9830 /10000 time/step/sample/link: 4.923e-05
step:  9831 /10000 time/step/sample/link: 5.573e-05
step:  9832 /

step:  9971 /10000 time/step/sample/link: 5.267e-05
step:  9972 /10000 time/step/sample/link: 5.148e-05
step:  9973 /10000 time/step/sample/link: 4.861e-05
step:  9974 /10000 time/step/sample/link: 8.015e-05
step:  9975 /10000 time/step/sample/link: 6.359e-05
step:  9976 /10000 time/step/sample/link: 5.871e-05
step:  9977 /10000 time/step/sample/link: 5.88e-05
step:  9978 /10000 time/step/sample/link: 8.171e-05
step:  9979 /10000 time/step/sample/link: 5.953e-05
step:  9980 /10000 time/step/sample/link: 5.952e-05
step:  9981 /10000 time/step/sample/link: 5.538e-05
step:  9982 /10000 time/step/sample/link: 6.836e-05
step:  9983 /10000 time/step/sample/link: 6.847e-05
step:  9984 /10000 time/step/sample/link: 5.45e-05
step:  9985 /10000 time/step/sample/link: 7.614e-05
step:  9986 /10000 time/step/sample/link: 5.982e-05
step:  9987 /10000 time/step/sample/link: 4.8e-05
step:  9988 /10000 time/step/sample/link: 4.936e-05
step:  9989 /10000 time/step/sample/link: 4.893e-05
step:  9990 /100

In [ ]:
model.sess.graph.collections

In [ ]:
model.sess.graph.get_collection

In [ ]:
model.dynamics.position_fn.summary()

In [ ]:
# Iterate over samples history and calculate observables for each sample.
# `lattice.calc_plaq_observables(samples)` calculates observables for each of
# the samples in the mini-batch.
actions_history = []
avg_plaquettes_history = []
top_charges_history = []
for idx, samples in enumerate(samples_history):
    t0 = time.time()
    observables = np.array(model.lattice.calc_plaq_observables(samples))
    actions, plaqs, charges = observables
    
    actions_history.append(actions)
    avg_plaquettes_history.append(plaqs)
    top_charges_history.append(charges)
    
    print(f'step: {idx}  '
          f'time / step: {time.time() - t0:^6.4g}  '
          f'avg action: {np.mean(actions):^6.4g}  '
          f'avg plaquette: {np.mean(plaqs):^6.4g} '
          f'top charge: {np.mean(charges):^6.4g}')

In [ ]:
_ = helpers.plot_run_data(model.data, 
                          model.params, 
                          model.steps_arr, 
                          model.figs_dir, 
                          skip_steps=1)

In [ ]:
#tf.reset_default_graph()
#model = GaugeModel(params=params,
#                   config=None,
#                   sess=None,
#                   conv_net=False,
#                   hmc=False,
#                   log_dir='../../gauge_logs_graph/run_25',
#                   restore=True)

In [ ]:
samples = np.random.randn(*model.samples.shape)
samples_history = []

In [ ]:
for i in range(500):
    t0 = time.time()
    samples = model.sess.run(model.x_out, feed_dict={model.x: samples})
    samples_history.append(samples)
    print(f'step: {i:^6.4g} time/step: {time.time() - t0:^6.4g}')

In [ ]:
samples_history_conv = np.array(samples_history_conv)
print(samples_history_conv.shape)

In [ ]:
import pickle
samples_history_file = os.path.join(model.info_dir, 'samples_history.pkl')
with open(samples_history_file, 'wb') as f:
    pickle.dump(samples_history_conv, f)